## 14 多路召回融合难？动态阈值机制确保高质量结果优先排序

## 一、多路召回与加权融合：从单一到多维的检索优化

### 1.1 传统做法的局限性

在传统的检索流程中，通常遵循“单一查询 → 向量检索 → 排序 → 生成答案”的模式。这种方法虽然直接，但在面对复杂或多意图查询时，其召回能力显得捉襟见肘，极易遗漏高相关但直接匹配度不高的文档。

### 1.2 优化做法的显著优势

为了大幅提升检索效果，优化的多路召回策略引入了多维度的检索范式：多个扩展查询 → 多次检索 → 聚合结果 → 重排序 → 生成答案

这种方法的核心在于：它不再局限于原始查询本身，而是通过查询扩展（Query Expansion）生成多个相关查询，并并行地在不同召回路径中进行搜索（例如，基于关键词的检索、基于向量相似度的检索、基于知识图谱的检索等）。这样做带来的显著优势包括：

- **提高召回覆盖率（Recall）**：通过从多角度、多数据源进行检索，能捕获更多与用户意图相关的文档，从而大大减少漏召的可能性。
- **利用 LLM 进行更精细的 Reranking**：将多路召回到的候选文档集合起来，再利用强大的**大型语言模型（LLM）**对这些候选段落进行更精细的相关性打分和排序。LLM凭借其强大的语义理解能力，能识别文档与查询之间更深层次的关联，显著提升最终排序的准确性。

---

## 二、伪代码示例：多路召回与 LLM 重排序

下面是一个简化的伪代码示例，展示了多路召回与 LLM 重排序的基本流程：

In [ ]:
retrieved_docs = []
for q in expanded_queries: # 遍历每个扩展查询
    docs = vector_db.search(q, top_k=5) # 从向量数据库中检索前 K 个文档
    retrieved_docs.extend(docs) # 将检索到的文档添加到集合中

# 使用 LLM 对所有召回的文档进行相关性评分并排序
reranked_docs = rerank_with_llm(retrieved_docs, original_query) 

## 二、引入基于置信度的动态融合阈值机制

尽管多路召回结合 LLM 重排序显著提升了检索效果，但在实际应用中，如何有效融合来自不同召回路径的结果，并确保高质量结果的优先排序，仍是一个挑战。尤其当不同召回路径返回结果质量不一或存在重复文档时，简单的加权融合可能无法达到最优效果。

### 2.1 核心思想

为解决这一难题，我们可以引入一种**基于置信度的动态融合阈值机制**。这种机制的核心思想是：

1. **评估召回路径的置信度**：  
   为每个召回路径的结果赋予一个置信度分数。这个分数可以基于多种因素确定，例如：
   - **匹配度得分**：召回模型返回的相似度分数或匹配度。
   - **路径的历史稳定性与准确性**：通过离线评估或在线 A/B 测试，统计不同召回路径在历史数据中的表现。
   - **召回文档的质量特征**：例如文档的来源权威性、内容的完整性等。

2. **动态调整融合权重**：  
   根据召回路径的置信度分数，自动调整不同召回路径的权重。置信度高的路径，其返回的结果在融合时将获得更高的权重。这意味着系统会更倾向于采信那些被认为更可靠、更高质量的召回源。

3. **设置动态阈值**：  
   引入一个动态阈值，只有达到一定置信度或相关性分数门槛的文档才会被纳入最终的重排序列表。这个阈值可以根据整体召回结果分布、查询复杂程度，甚至是用户历史行为动态调整。例如，当召回的整体质量普遍较高时，可适当提高阈值以筛选更优质结果；反之，在召回结果较少或质量不佳时，可适当降低阈值以确保一定的召回数量。

---

### 2.2 引入机制的目标

通过引入基于置信度的动态融合阈值机制，我们能实现以下关键目标：

- ✅ **确保高质量结果优先排序**：  
  高置信度召回路径中的高质量文档将获得更高的融合权重，并在动态阈值的筛选下优先进入 LLM 重排序阶段。这意味着系统能够更有效地将真正相关的、有价值的信息推到用户面前。

- ✅ **提升排序稳定性**：  
  动态调整权重和阈值，使得系统能更好地适应不同查询类型和数据分布。即使某些召回路径在特定情况下表现不佳，整体融合机制也能通过降低其权重，避免其对最终排序结果产生负面影响，从而提升整个检索系统的稳定性。

---

### 2.3 演示代码：多路召回与 LLM 重排序

为了更好地理解上述概念，这里提供一个可运行的 Python 演示代码。该代码使用 [LlamaIndex](https://www.llamaindex.ai/ ) 框架与 OpenAI 大模型进行交互，展示了多路召回、文档去重以及重排序过程。


In [1]:
import os
import math
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever

from pydantic import ConfigDict

/Users/linghuang/miniconda3/envs/llm_clean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [9]:
# -----------------------------------------------------------------------------
# 1) Initialization
# -----------------------------------------------------------------------------
# Make sure OPENAI_API_KEY is set:
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Sample documents (same meaning as your LlamaIndex demo)
docs = [
    Document(
        page_content="Artificial Intelligence (AI) is a new technical science that aims to simulate, extend, and augment human intelligence through theories, methods, technologies, and application systems.",
        metadata={"source": "wiki"},
    ),
    Document(
        page_content="Deep learning is a new field within machine learning research, motivated by the construction of neural networks that simulate the human brain for analytical learning.",
        metadata={"source": "wiki"},
    ),
    Document(
        page_content="LLM stands for Large Language Model. It is an artificial intelligence model trained on large volumes of text data and is widely used in natural language processing tasks.",
        metadata={"source": "tech_blog"},
    ),
]

# Vector index (FAISS) + embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# Keyword/BM25 retriever
bm25 = BM25Retriever.from_documents(docs)
bm25.k = 2

In [10]:
from typing import List, Tuple, Any
import math

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from pydantic import ConfigDict


class MultiPathRetriever(BaseRetriever):
    # ✅ Declare fields so Pydantic allows assignment
    bm25_retriever: Any
    vectorstore: Any

    vector_top_k: int = 2
    bm25_top_k: int = 2
    vector_weight: float = 0.8
    bm25_weight: float = 0.6

    # ✅ Allow non-Pydantic types (FAISS, BM25Retriever)
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # --- BM25 path ---
        try:
            self.bm25_retriever.k = self.bm25_top_k
        except Exception:
            pass
        bm25_docs = self.bm25_retriever.invoke(query)

        # assign a simple rank-based score (no native BM25 scores exposed)
        bm25_scored = [
            (d, (1.0 / (i + 1)) * self.bm25_weight, "bm25")
            for i, d in enumerate(bm25_docs)
        ]

        # --- Vector path (FAISS) ---
        # similarity_search_with_score returns (doc, distance) where lower is better
        faiss_with_scores: List[Tuple[Document, float]] = self.vectorstore.similarity_search_with_score(
            query, k=self.vector_top_k
        )
        vec_scored = [
            (d, (1.0 / (1.0 + dist)) * self.vector_weight, "vector")
            for d, dist in faiss_with_scores
        ]

        # --- Merge + dedup by content ---
        best = {}
        for d, s, src in (vec_scored + bm25_scored):
            key = d.page_content
            if key not in best or s > best[key][1]:
                best[key] = (d, s, src)

        merged = list(best.values())
        if not merged:
            return []

        # --- Dynamic threshold: mean + std (keep at least 1) ---
        scores = [s for _, s, _ in merged]
        mean = sum(scores) / len(scores)
        std = math.sqrt(sum((s - mean) ** 2 for s in scores) / len(scores))
        thr = mean + std

        merged.sort(key=lambda x: x[1], reverse=True)
        kept = [x for x in merged if x[1] >= thr] or merged[:1]

        # add scores to metadata
        out = []
        for d, s, src in kept:
            md = dict(d.metadata or {})
            md["fusion_score"] = float(s)
            md["retrieval_path"] = src
            out.append(Document(page_content=d.page_content, metadata=md))
        return out


In [12]:
from langchain_community.retrievers import BM25Retriever
import Stemmer

stemmer = Stemmer.Stemmer("english")
bm25_retriever = BM25Retriever.from_documents(docs, k=2, stemmer=stemmer)

def preview(doc: Document, n: int = 500) -> str:
    text = (doc.page_content or "").replace("\n", " ").strip()
    return text[:n] + ("..." if len(text) > n else "")

def print_results(title: str, docs: List[Document], n_preview: int = 500) -> None:
    print(f"\n{title} (count={len(docs)})")
    for i, d in enumerate(docs, 1):
        src = (d.metadata or {}).get("source", "")
        print("-" * 80)
        print(f"[{i}] source={src}")
        print(preview(d, n=n_preview))

retriever = MultiPathRetriever(
    vectorstore=vectorstore,          # NOTE: pass the FAISS *vectorstore*, not as_retriever()
    bm25_retriever=bm25_retriever,
    vector_top_k=2,
    bm25_top_k=2,
    vector_weight=0.8,
    bm25_weight=0.6,
)

query = "What happened at Viaweb and Interleaf?"
hybrid_docs = retriever.invoke(query)   # ✅ LangChain 1.x way
print_results("Hybrid Results", hybrid_docs)

: 

In [36]:
# -----------------------------------------------------------------------------
# 3) LLM reranking (OpenAI via LangChain)
# -----------------------------------------------------------------------------

def llm_rerank(query: str, docs: List[Document], llm: ChatOpenAI) -> List[Document]:
    """
    Ask the LLM for a 0-10 relevance score for each doc, then sort descending.
    Stores LLM score in doc.metadata["llm_rerank_score"].
    """
    if not docs:
        return []

    scored_docs: List[Tuple[Document, float]] = []

    for d in docs:
        prompt = (
            "You are a strict relevance grader.\n"
            f"Question: {query}\n\n"
            "Passage:\n"
            f"{d.page_content}\n\n"
            "Return ONLY a single number between 0 and 10."
        )

        resp = llm.invoke(prompt).content.strip()

        try:
            score = float(resp)
        except ValueError:
            # fallback if model returns something unexpected
            score = 0.0

        d.metadata["llm_rerank_score"] = score
        scored_docs.append((d, score))

    scored_docs.sort(key=lambda x: x[1], reverse=True)
    return [d for d, _ in scored_docs]

In [37]:
vectorstore

In [39]:
# -----------------------------------------------------------------------------
# 4) Run demo
# -----------------------------------------------------------------------------
retriever = MultiPathRetriever(
    vectorstore=vectorstore,
    bm25_retriever=bm25,
    vector_top_k=2,
    bm25_top_k=2,
    vector_weight=0.8,
    bm25_weight=0.6,
)

query = "What is a large language model?"
candidate_docs = retriever.invoke(query)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # pick any OpenAI chat model you have access to
reranked_docs = llm_rerank(query, candidate_docs, llm)

print("--- Final Retrieved and Re-Ranked Documents ---")
if not reranked_docs:
    print("No relevant documents were found.")
else:
    for i, d in enumerate(reranked_docs, start=1):
        rs = d.metadata.get("retrieval_score", None)
        path = d.metadata.get("retrieval_path", None)
        llm_s = d.metadata.get("llm_rerank_score", None)
        print(f"{i}. {d.page_content}")
        print(f"   retrieval_score={rs:.3f} path={path} | llm_rerank_score={llm_s:.1f}")

: 

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex, Document
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from typing import List
from openai import OpenAI
import os

# -----------------------------------------------------------------------------
# 1. 初始化设置
# -----------------------------------------------------------------------------

# 初始化 OpenAI 客户端
# 请确保您的 OPENAI_API_KEY 已经设置为环境变量
# 例如: os.environ["OPENAI_API_KEY"] = "sk-..."
client = OpenAI()

# 准备用于演示的原始文档
# 这里的文档内容很简单，仅用于展示流程
documents = [
    Document(text="人工智能（AI）是一种旨在模拟、延伸和扩展人类智能的理论、方法、技术及应用系统的一门新的技术科学。", metadata={"source": "wiki"}),
    Document(text="深度学习是机器学习研究中的一个新的领域，其动机在于建立、模拟人脑进行分析学习的神经网络。", metadata={"source": "wiki"}),
    Document(text="LLM 是大型语言模型的缩写，它是一种经过大量文本数据训练的人工智能模型，广泛用于自然语言处理任务。", metadata={"source": "tech_blog"}),
]

# 基于文档，分别构建两种不同类型的索引：向量索引和关键词索引
# 向量索引擅长理解语义相关性（意思相近）
vector_index = VectorStoreIndex.from_documents(documents)
# 关键词索引擅长精确匹配文本中的词语
keyword_index = SimpleKeywordTableIndex.from_documents(documents)


# -----------------------------------------------------------------------------
# 2. 自定义多路召回的Retriever
# -----------------------------------------------------------------------------

class MultiPathRetriever(BaseRetriever):
    """
    自定义一个融合了向量搜索和关键词搜索的混合检索器。
    这种方法结合了两种搜索方式的优点，可以提高召回文档的全面性和准确性。
    """
    def __init__(self, vector_retriever, keyword_retriever):
        self.vector_retriever = vector_retriever
        self.keyword_retriever = keyword_retriever
        super().__init__()

    def _retrieve(self, query: str) -> List[NodeWithScore]:
        # 第一步：并行从两个检索器中获取结果
        vector_nodes = self.vector_retriever.retrieve(query)
        keyword_nodes = self.keyword_retriever.retrieve(query)

        # 第二步：为不同来源的结果分配不同的权重，并合并
        # 这里的权重是经验值，可以根据实际效果调整。我们假设向量检索的结果更可靠一些。
        for node in vector_nodes:
            node.score = (node.score or 1.0) * 0.8  # 为向量检索结果分配0.8的权重
        for node in keyword_nodes:
            node.score = (node.score or 1.0) * 0.6  # 为关键词检索结果分配0.6的权重

        combined_nodes = vector_nodes + keyword_nodes

        # 第三步：去重
        # 将两个路径返回的节点合并后，可能会有重复的文档，需要基于文本内容进行去重。
        seen_texts = set()
        unique_nodes = []
        for node in combined_nodes:
            if node.text not in seen_texts:
                seen_texts.add(node.text)
                unique_nodes.append(node)
        
        # 第四步：动态阈值过滤
        # 为了只保留最相关的文档，我们计算一个动态阈值（平均分+标准差），过滤掉得分较低的文档。
        scores = [n.score for n in unique_nodes if n.score is not None]
        if not scores:
            return []  # 如果没有任何带分数的节点，则返回空

        mean_score = sum(scores) / len(scores)
        std_dev = (sum((s - mean_score)**2 for s in scores) / len(scores))**0.5
        threshold = mean_score + std_dev

        # 只返回得分高于阈值的文档
        filtered_nodes = [n for n in unique_nodes if n.score is not None and n.score >= threshold]

        return filtered_nodes

# -----------------------------------------------------------------------------
# 3. 执行检索与重排序
# -----------------------------------------------------------------------------

# 实例化我们的多路召回检索器
# similarity_top_k=2 表示每个检索器最多返回2个最相关的结果
retriever = MultiPathRetriever(
    vector_retriever=vector_index.as_retriever(similarity_top_k=2),
    keyword_retriever=keyword_index.as_retriever(similarity_top_k=2)
)

# 定义一个查询问题
query = "什么是大型语言模型？"
# 执行检索，获取经过初步筛选和过滤的文档节点
nodes = retriever.retrieve(query)

# LLM 重排序 (Reranking)
# 这是提升最终结果质量的关键一步。
# 我们让一个强大的语言模型（如GPT）来评估每个召回的文档与原始问题的相关性，并给出一个分数。
# 然后根据这个分数对文档进行重新排序。
reranked_nodes = []
if nodes:
    reranked_nodes = sorted(nodes, key=lambda x: float(client.completions.create(
        model="gpt-3.5-turbo-instruct",  # 使用一个适合做指令跟随任务的模型
        prompt=f"请评估以下段落与问题 '{query}' 的相关性，仅需给出一个0到10之间的分数。\n\n段落：{x.text}\n\n相关性评分：",
        max_tokens=5,  # 只需要输出一个分数，所以限制token数量
        temperature=0  # 为了让评分更稳定，使用确定性输出
    ).choices[0].text.strip() or "0"), reverse=True)


# -----------------------------------------------------------------------------
# 4. 输出最终结果
# -----------------------------------------------------------------------------

print("--- 最终召回并重排序的文档 ---")
if not reranked_nodes:
    print("未能找到相关的文档。")
else:
    for i, node in enumerate(reranked_nodes):
        # 打印出重排序后的文档内容和它在召回阶段的原始分数
        print(f"{i+1}. {node.text} (召回分数: {node.score:.2f})")



--- 最终召回并重排序的文档 ---
1. LLM 是大型语言模型的缩写，它是一种经过大量文本数据训练的人工智能模型，广泛用于自然语言处理任务。 (召回分数: 0.69)


基于 LLM 的重排序过程

In [ ]:
import os
from typing import List, Dict, Any
from llama_index.core import Document, QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore # 导入 NodeWithScore
from llama_index.llms.openai import OpenAI as LlamaOpenAI # 导入LlamaIndex的OpenAI LLM封装，并重命名以示区分

# --- 1. 环境与配置 ---

# 确保OPENAI_API_KEY环境变量已设置
# 在实际使用中，请将 "YOUR_OPENAI_API_KEY" 替换为您的真实密钥
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("请设置 OPENAI_API_KEY 环境变量。")

# --- 2. 模拟召回器 (用于演示) ---

class MockRetriever(BaseRetriever):
    """
    一个模拟的检索器，用于演示目的。
    在真实场景中，这里应该是来自不同来源的多个真实检索器，
    例如：向量数据库检索器、关键词检索器、图数据库检索器等。
    """
    def __init__(self, all_docs: List[Document]):
        self._all_docs = all_docs
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        模拟检索过程。
        为了简化，这里仅通过简单的关键词匹配来返回文档。
        """
        query_text = query_bundle.query_str.lower()
        retrieved_nodes = []
        for doc in self._all_docs:
            # 如果查询中的任何一个词出现在文档中，就认为召回成功
            if any(keyword in doc.text.lower() for keyword in query_text.split()):
                # --- 修复 ---
                # LlamaIndex的Retriever标准输出格式是NodeWithScore对象列表。
                # 此处将召回的Document包装成NodeWithScore，以符合框架要求。
                retrieved_nodes.append(NodeWithScore(node=doc, score=1.0))
        # 模拟真实检索器返回 top_k 个结果
        return retrieved_nodes[:5]

# --- 3. 核心功能：多路召回与LLM重排序 ---

def retrieve_and_rerank(
    original_query: str,
    expanded_queries: List[str],
    all_docs: List[Document]
) -> List[Dict[str, Any]]:
    """
    执行多路召回，聚合结果，然后使用 LLM 进行重排序。

    Args:
        original_query (str): 用户的原始查询。
        expanded_queries (List[str]): 由原始查询扩展出的一组子查询。
        all_docs (List[Document]): 知识库中的所有文档。

    Returns:
        List[Dict[str, Any]]: 经过重排序并带有分数的文档列表。
    """
    print(f"原始查询: '{original_query}'")
    print(f"扩展查询: {expanded_queries}\n")

    # --- 步骤 1: 多路召回与结果聚合 ---
    # 初始化模拟检索器
    retriever = MockRetriever(all_docs)
    
    retrieved_docs = {}  # 使用字典去重，键为文档ID，值为文档对象
    for q in expanded_queries:
        print(f"--- 正在执行召回, 查询: '{q}' ---")
        # 从模拟检索器获取文档
        nodes_from_path = retriever.retrieve(QueryBundle(query_str=q))
        print(f" -> 召回了 {len(nodes_from_path)} 篇文档。")
        for node_with_score in nodes_from_path:
            doc = node_with_score.node
            retrieved_docs[doc.id_] = doc # 利用字典特性自动去重

    unique_docs = list(retrieved_docs.values())
    print(f"\n多路召回共聚合了 {len(unique_docs)} 篇不重复的文档。\n")

    if not unique_docs:
        return []

    # --- 步骤 2: LLM 重排序 ---
    print("--- 开始使用 LLM 对召回的文档进行重排序 ---")
    
    # 初始化 LlamaIndex 的 LLM 封装
    llm = LlamaOpenAI(model="gpt-4o", temperature=0.0)
    
    reranked_results = []
    for doc in unique_docs:
        # 为每个文档构建一个用于评分的提示 (Prompt)
        prompt = (
            f"评估以下文档内容与用户查询的相关性。请只输出一个0到100之间的整数分数，100表示最相关。\n\n"
            f"用户查询: '{original_query}'\n\n"
            f"文档内容: '{doc.text}'\n\n"
            f"相关性分数:"
        )
        try:
            # 调用 LLM 获取评分
            response = llm.complete(prompt)
            score = int(response.text.strip())
            print(f"  - 正在评分: 文档ID '{doc.id_}' ... 分数: {score}")
            reranked_results.append({"doc": doc, "score": score})
        except (ValueError, AttributeError) as e:
            print(f"  - 评分失败: 文档ID '{doc.id_}', 错误: {e}。记为0分。")
            reranked_results.append({"doc": doc, "score": 0})

    # 根据 LLM 给出的分数进行降序排序
    reranked_results.sort(key=lambda x: x["score"], reverse=True)

    return reranked_results

# --- 4. 演示执行 ---

if __name__ == "__main__":
    # 模拟知识库中的文档
    mock_docs = [
        Document(id_="doc1", text="Python是一种广泛使用的编程语言，常用于数据科学和机器学习。"),
        Document(id_="doc2", text="机器学习是人工智能的一个分支，专注于让计算机从数据中学习。"),
        Document(id_="doc3", text="如何使用LlamaIndex进行文档索引和检索？LlamaIndex是一个用于将LLM与外部数据连接的数据框架。"),
        Document(id_="doc4", text="自然语言处理(NLP)是人工智能的另一个领域，涉及计算机理解和生成人类语言。"),
        Document(id_="doc5", text="Python的语法非常简洁和易读，适合初学者入门。"),
        Document(id_="doc6", text="大型语言模型(LLM)是近年来人工智能领域的重要突破，它们能够生成连贯且相关的文本。"),
        Document(id_="doc7", text="深度学习是机器学习的一个子集，使用神经网络进行数据建模。"),
        Document(id_="doc8", text="使用OpenAI API可以访问各种强大的LLM模型，进行文本生成、摘要和问答等任务。"),
    ]

    # 原始查询 和 手动扩展的子查询
    original_query = "关于LlamaIndex和大型语言模型的信息"
    expanded_queries = [
        "LlamaIndex 如何工作?",
        "大型语言模型应用",
        "LlamaIndex 和 LLM",
        "OpenAI API 使用",
    ]

    # 执行召回和重排序流程
    final_results = retrieve_and_rerank(original_query, expanded_queries, mock_docs)

    # 打印最终排序结果
    print("\n--- 最终重排序结果 (按LLM评分降序) ---")
    if not final_results:
        print("未能找到任何相关文档。")
    else:
        for i, item in enumerate(final_results):
            doc = item['doc']
            score = item['score']
            print(f"{i+1}. [分数: {score}] (ID: {doc.id_}) 内容: '{doc.text[:100]}...'")


原始查询: '关于LlamaIndex和大型语言模型的信息'
扩展查询: ['LlamaIndex 如何工作?', '大型语言模型应用', 'LlamaIndex 和 LLM', 'OpenAI API 使用']

--- 正在执行召回, 查询: 'LlamaIndex 如何工作?' ---
 -> 召回了 1 篇文档。
--- 正在执行召回, 查询: '大型语言模型应用' ---
 -> 召回了 0 篇文档。
--- 正在执行召回, 查询: 'LlamaIndex 和 LLM' ---
 -> 召回了 5 篇文档。
--- 正在执行召回, 查询: 'OpenAI API 使用' ---
 -> 召回了 4 篇文档。

多路召回共聚合了 7 篇不重复的文档。

--- 开始使用 LLM 对召回的文档进行重排序 ---
  - 正在评分: 文档ID 'doc3' ... 分数: 85
  - 正在评分: 文档ID 'doc1' ... 分数: 0
  - 正在评分: 文档ID 'doc4' ... 分数: 20
  - 正在评分: 文档ID 'doc5' ... 分数: 0
  - 正在评分: 文档ID 'doc6' ... 分数: 80
  - 正在评分: 文档ID 'doc7' ... 分数: 0
  - 正在评分: 文档ID 'doc8' ... 分数: 60

--- 最终重排序结果 (按LLM评分降序) ---
1. [分数: 85] (ID: doc3) 内容: '如何使用LlamaIndex进行文档索引和检索？LlamaIndex是一个用于将LLM与外部数据连接的数据框架。...'
2. [分数: 80] (ID: doc6) 内容: '大型语言模型(LLM)是近年来人工智能领域的重要突破，它们能够生成连贯且相关的文本。...'
3. [分数: 60] (ID: doc8) 内容: '使用OpenAI API可以访问各种强大的LLM模型，进行文本生成、摘要和问答等任务。...'
4. [分数: 20] (ID: doc4) 内容: '自然语言处理(NLP)是人工智能的另一个领域，涉及计算机理解和生成人类语言。...'
5. [分数: 0] (ID: doc1) 内容: 'Python是一种广泛使用的编程语言，常用于数据科学和机器学习。...'
6. [分

## 总结

多路召回结合 LLM 重排序是提升信息检索系统性能的强大策略。

而基于置信度的动态融合阈值机制则能够有效解决多路召回结果融合的难题，更能确保高质量结果的优先排序，并显著提升整个系统的稳定性和鲁棒性。